In [2]:
a = ["a","b","c"]
b = [2,1,3]


exclude = ["b"]


a_cut = []
b_cut = []

for pair in zip(a,b):
    if pair[0] not in exclude:
        a_cut.append(pair[0])
        b_cut.append(pair[1])

print(a_cut)
print(b_cut)

['a', 'c']
[2, 3]


In [3]:
expr = "x+y"
values = {"x":1, "y":5}
result = eval(expr, {}, values)
print(result) # Output: 6


[1, 3, 5, 7]


In [2]:
import numpy as np
A = np.full((3,3),0.)
A = np.zeros((3,3))
idx = (0,0)
A[idx] = np.nan
A

array([[nan,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [6]:
import numpy as np

d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = {'b': 4, 'a': 5, 'c': 6}

d3 = {k: d2[k] for k in d1}
d3
tuple(d2[k] for k in d1)

(5, 4, 6)

In [78]:
class Test:
    def __init__(self):
        pass

    def __mul__(self,other):
        return 3
    
    


a = np.array([(3,4)])
b = np.array([(5,6)])

a@b

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2)

In [88]:
class Unknown:
    def __init__(self):
        pass

    @staticmethod
    def __multiply(value):
        if value==0:
            return 0
        else:
            return Unknown()
                
    def __add():
        return Unknown()
        


    def __mul__(self,other):
        return Unknown.__multiply(other)

    def __rmul__(self,other):
        return Unknown.__multiply(other)


    def __add__(self,other):
        return Unknown.__add()
    
    def __radd__(self,other):
        return Unknown.__add()
    
    def __repr__(self):
        return "?"
    

print(Unknown()*0*Unknown()+Unknown())

?


In [25]:
import scipy
from scipy import signal

def kernel(derivative,order):
    FD1_1 = [0,-1,1]
    BD1_1 = [-1,1,0]
    CD1_2 = [-0.5,0,0.5]



    def convolve(k1,k2):
        return scipy.signal.convolve(k1,k2)


    if order>2

    CD2_2 = convolve(FD1_1,BD1_1)

    FD2_2 = convolve(FD1_1,FD1_1)

    convolve(BD1_1,convolve(FD1_1,CD2_2))

array([ 0,  0,  1, -4,  6, -4,  1,  0,  0])

In [26]:
(3,4)==(3,4)

True